### POLITICAL Dataset for California:
* import
* cleaning
* exploration with maps

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Bring in population data by County in CA: 
* reformat and clean data to prep for merge on 'County'

In [17]:
#need population of counties to see cases as a % of population
population_df = pd.read_html('https://www.california-demographics.com/counties_by_population', index_col = 0)[0].iloc[:-1,:]

#remove the word 'county' in column county, and save to folder
population_df['County'] = population_df['County'].str.replace(' County','')
population_df.to_csv('../data/population_df.csv')
population_df.head(1)

,County,Population
Rank,,
1,Los Angeles,10039107


## 2. Bring in votes by County in CA:
* remove NaN values
* remove extra formatted rows not needed ie. "percentage"
* remove all candidates except for major: Hilary Clinton, Bernie Sanders, Donald Trump

In [18]:
#read in data excluding Percent, NaN, and last row
political_df = pd.read_excel('../data/California_president_county.xls', index_col = 0).iloc[1::3].iloc[:-1,:]
political_df.head(1)

,Hillary Clinton,Bernie Sanders,Roque\nDe La Fuente,Henry Hewes,Keith\nJudd,Michael Steinberg,Willie Wilson,Kevin M. Moreau,Willie Felix Carter,Andrew D. Basiago,...,Darryl W. Perry,Austin Petersen,Derrick M. Reid,Rhett White Feather Smith,Joy Waymire,John \nHale,"Jack Robinson, Jr.",Gloria Estela La Riva,Lynn S. Kahn,Monica Moorehead
Alameda,164889,153955,249,322,211,333,462,0,0,0,...,17,44,17,69,29,41,30,135,39,86


In [19]:
#drop NaNs
political_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
#political_df.head(3)

In [20]:
# rename columns to clean up names for candidates we want to keep
political_df = political_df.rename(columns={'Ted \nCruz': 'Ted Cruz', 'Ben\nCarson': 'Ben Carson'})
political_df['Ben Carson'].head(1)

Alameda    1973
Name: Ben Carson, dtype: object

### 3. Create New DF with candidates that make up 99% of the votes for each political party

In [21]:
major_candidates_df = political_df[['Hillary Clinton', 'Bernie Sanders', 'Donald Trump', 'John R. Kasich', 'Ted Cruz', 'Ben Carson']].copy()
#major_candidates_df.head(1)

In [22]:
# add total vote categories by Democrats and Republicans
major_candidates_df['Republican Votes'] = major_candidates_df['Donald Trump'] + major_candidates_df['John R. Kasich'] + major_candidates_df['Ted Cruz'] + major_candidates_df['Ben Carson']
major_candidates_df['Democratic Votes'] = major_candidates_df['Hillary Clinton'] + major_candidates_df['Bernie Sanders']
#major_candidates_df.head(3)

In [23]:
#change the index, name the County Column
major_candidates_df.reset_index(inplace=True)
major_candidates_df.rename(columns = {'index': 'County'}, inplace=True)

In [24]:
# add political affiliation by % of population
major_candidates_df['%_Republican'] = major_candidates_df['Republican Votes']/(major_candidates_df['Democratic Votes'] + major_candidates_df['Republican Votes']) *100
major_candidates_df['%_Democrat'] = major_candidates_df['Democratic Votes']/(major_candidates_df['Democratic Votes'] + major_candidates_df['Republican Votes']) *100
#major_candidates_df.head(3)

In [25]:
#assign political affiliations
import sys
sys.path.insert(0, '/Users/megan/Galvanize/capstone_folder/Capstone_1')
from src.scripts import repub_or_dem

repub_or_dem(major_candidates_df)

,County,Hillary Clinton,Bernie Sanders,Donald Trump,John R. Kasich,Ted Cruz,Ben Carson,Republican Votes,Democratic Votes,%_Republican,%_Democrat,Affiliation,Aff_Code
0,Alameda,164889,153955,29097,7614,4807,1973,43491,318844,12.003,87.997,Democrat,1
1,Alpine,113,137,123,19,7,5,154,250,38.1188,61.8812,Democrat,1
2,Amador,2516,2386,4868,478,399,217,5962,4902,54.8785,45.1215,Republican,0
3,Butte,11766,19739,20079,1828,1928,1231,25066,31505,44.3089,55.6911,Democrat,1
4,Calaveras,2971,2914,5763,499,513,209,6984,5885,54.27,45.73,Republican,0


In [26]:
#save here
major_candidates_df.to_csv('../data/major_candidates_df.csv')

### 4. Map the counties by % Political Affiliation

In [29]:
import folium

In [30]:
# Political Affiliation Map
choroMap = folium.Map(location=(36.78,-119.42), zoom_start=6, control_scale = True) #instantiate blank map, chose 'california coord' to start
choroMap.choropleth(geo_data='../data/california_counties.geojson',
                    data = major_candidates_df,
                    columns = ['County', '%_Democrat'],
                    key_on = 'feature.properties.name',
                    fill_color = 'RdYlBu', #'YlOrRd',
                    fill_opacity = 0.8,
                    line_opacity = 0.8,
                    legend_name = 'Political Affiliation by County')

choroMap.save('../images/%_Political_Map_2.png')
choroMap

In [31]:
Dem_Rep_Map = folium.Map(location=(36.78,-119.42), zoom_start=6, control_scale = True) #instantiate blank map, chose 'california coord' to start
Dem_Rep_Map.choropleth(geo_data='../data/california_counties.geojson',
                    data = major_candidates_df,
                    columns = ['County', "Aff_Code"],
                    key_on = 'feature.properties.name',
                    fill_color = 'RdBu', #'YlOrRd',
                    fill_opacity = 0.8,
                    line_opacity = 0.8,
                    legend_name = 'Political Affiliation by County')
Dem_Rep_Map.save('../images/Dem_Rep_Map_1.html') 
Dem_Rep_Map